In [6]:
import pandas as pd
import json
import requests

city_bikes_data = pd.read_csv("C:/Users/colte/LHL-final-project/data/stations_df.csv")

    city country                       name   latitude  longitude  slots
0  Turku      FI               Asesepänkatu  60.441012  22.256590      8
1  Turku      FI  Runosmäki Signalistinkatu  60.485777  22.268321      8
2  Turku      FI       Sale Kupittaanpuisto  60.448001  22.288602      3
3  Turku      FI          Sairashuoneenkatu  60.448961  22.246354      6
4  Turku      FI     Citymarket Länsikeskus  60.472613  22.233105     10


In [ ]:
# Set latitude and longitude for params
latitude = city_bikes_data['latitude']
longitude = city_bikes_data['longitude']

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [99]:
# Foursquare API Key
FOURSQUARE_KEY = 'fsq3nRLHnLQh2Y02PDBx8qcYaOIKIzeNRUXditJZC30vLIU='

# Foursquare url
foursquare_url = "https://api.foursquare.com/v3/places/search"

# Foursquare categories for params
foursquare_categories = "13065,13003,17043,17048"

# Foursquare header
foursquare_headers = {
    "accept": "application/json",
    "Authorization": f"{FOURSQUARE_KEY}"
}

# Foursquare params
foursquare_params = {
    "ll": f"{latitude},{longitude}",
    "radius": 1000,
    "limit": 10,
    "categories": foursquare_categories,
    "sort": "DISTANCE"
}

foursquare_response = requests.get(foursquare_url, headers=foursquare_headers, params=foursquare_params)

print(foursquare_response.text)

{"results":[{"fsq_id":"4b93ec56f964a5201c5a34e3","categories":[{"id":13003,"name":"Bar","short_name":"Bar","plural_name":"Bars","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/nightlife/pub_","suffix":".png"}},{"id":13057,"name":"Gastropub","short_name":"Gastropub","plural_name":"Gastropubs","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/gastropub_","suffix":".png"}}],"chains":[],"closed_bucket":"LikelyOpen","distance":249,"geocodes":{"main":{"latitude":60.439055,"longitude":22.259166},"roof":{"latitude":60.439055,"longitude":22.259166}},"link":"/v3/places/4b93ec56f964a5201c5a34e3","location":{"address":"Stålarminkatu 2","country":"FI","cross_street":"","formatted_address":"Stålarminkatu 2, 20810 Turku","locality":"Turku","postcode":"20810","region":"Varsinais-Suomi"},"name":"Kortteliravintola Hugo","related_places":{},"timezone":"Europe/Helsinki"},{"fsq_id":"5108fdade4b0f42b072b1bdd","categories":[{"id":17043,"name":"Clothing Store","short_name":"Apparel","plur

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [105]:
# Put response data into a dict
foursquare_data = foursquare_response.json()

# Find the appropriate fields for business name, category, address, and rating
for business in foursquare_data.get("results", []):
    name = business.get("name")
    categories = [category["short_name"] for category in business.get("categories", [])]
    rating = business.get("rating", 'N/A')
    address = business.get("location", {}).get("address", "N/A")
    print(f"name: {name}, address: {address}, rating: {rating}, categories: {', '.join(categories)}")

name: Kortteliravintola Hugo, address: Stålarminkatu 2, rating: N/A, categories: Bar, Gastropub
name: Vaate & Paikka, address: N/A, rating: N/A, categories: Apparel
name: Kasperin Yksiö, address: N/A, rating: N/A, categories: Hotel Bar


Put your parsed results into a DataFrame

In [111]:
# Extract business list
foursquare_businesses = foursquare_data.get("results", [])

#Create list of required fields from the foursquare dict
foursquare_list = [
    {
        "name": business.get("name"),
        "address": business.get("location", {}).get("address", "N/A"),
        "categories": ", ".join([category["short_name"] for category in business.get("categories", [])]),
        "rating": business.get("rating", None),
        "latitude": business.get("geocodes", {}).get("main", {}).get("latitude", "N/A"),
        "longitude": business.get("geocodes", {}).get("main", {}).get("longitude", "N/A")
    }
    for business in foursquare_businesses
]

# Convert list to df
foursquare_df = pd.DataFrame(foursquare_list)

# Create file path for foursquare df
foursquare_file_path = "C:/Users/colte/LHL-final-project/data/foursquare_df.csv"

# Add foursquare_df.csv to file path
foursquare_df.to_csv(foursquare_file_path, index=False)

print(f"df was saved to {foursquare_file_path}")

df was saved to C:/Users/colte/LHL-final-project/data/foursquare_df.csv


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [76]:
# Yelp API Key
YELP_KEY = 'ny8E6_YvMJMaxR8895q-m4dxWtNwB0HaxfYWA78EJ0xKTlhPHTo6hbN-oqeDea6SYJcR6xHtBGnmXm5QH8n3ohjlg63LkCG5BGgVkJppyyLJ1N08bdYbCzsTGPtYZ3Yx'

# Yelp url
yelp_url = "https://api.yelp.com/v3/businesses/search"

# Yelp categories for params
yelp_categories = "restaurants, bars, streetwear, sneakers, clothing, shoe"

#Yelp header
yelp_headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {YELP_KEY}"
}

yelp_params = {
    "sort_by": "distance",
    "limit": 10,
    "radius": 1000,
    "latitude": latitude,
    "longitude": longitude,
    "categories": yelp_categories
}

response = requests.get(yelp_url, headers=yelp_headers, params=yelp_params)

print(response.text)

{"businesses": [{"id": "YgoYUU_2snBOLnJlS1Fkew", "alias": "pub-hopotus-turku", "name": "Pub Hopotus", "image_url": "", "is_closed": false, "url": "https://www.yelp.com/biz/pub-hopotus-turku?adjust_creative=csjvSJ0AyxJsx4avWsfL1Q&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=csjvSJ0AyxJsx4avWsfL1Q", "review_count": 0, "categories": [{"alias": "restaurants", "title": "Restaurants"}], "rating": 0.0, "coordinates": {"latitude": 60.4394493, "longitude": 22.2590599}, "transactions": [], "location": {"address1": "Kuninkaankartanonkatu 8", "address2": null, "address3": null, "city": "Turku", "zip_code": "20810", "country": "FI", "state": "19", "display_address": ["Kuninkaankartanonkatu 8", "20810 Turku", "Finland"]}, "phone": "", "display_phone": "", "distance": 189.28231097649208, "business_hours": [], "attributes": {"business_temp_closed": null, "menu_url": null, "open24_hours": null, "waitlist_reservation": null}}, {"id": "ds8wMeO071xupvBoMtMZWg", "alias": "martinm\u

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [78]:
# Put response data into a dict
yelp_data = response.json()

# Find the appropriate fields for business name, category, address, and rating
for business in yelp_data.get("businesses", []):
    name = business.get("name")
    categories = [category["title"] for category in business.get("categories", [])]
    rating = business.get("rating", 'N/A')
    address = business.get("location", {}).get("address1", "")
    print(f"name: {name}, address: {address}, rating: {rating}, categories: {', '.join(categories)}")

name: Pub Hopotus, address: Kuninkaankartanonkatu 8, rating: 0.0, categories: Restaurants
name: Martinmäen kotiruokapalvelu Avoin, address: Tiilentekijänkatu 10J, rating: 0.0, categories: Restaurants
name: Naantalin Aurinkoinen, address: Martinkatu 3, rating: 0.0, categories: Restaurants
name: Melange, address: Martinkatu 3, rating: 0.0, categories: Restaurants
name: Maku Turku, address: Stålarminkatu 5, rating: 2.5, categories: Pan Asian
name: Ravintola Pippurimylly, address: Stålarminkatu 2, rating: 4.7, categories: Steakhouses
name: designed by oodi pr, address: Tiilentekijänkatu 3B A, rating: 0.0, categories: Restaurants
name: Haarala Tiina, address: Itäinen Rantakatu 48H 242, rating: 0.0, categories: Restaurants
name: Ravintola-Pizzeria Ranta-Osku avoin, address: Martinkatu 1, rating: 0.0, categories: Restaurants
name: T:mi Maryam-M, address: Martinkatu 1, rating: 0.0, categories: Restaurants


Put your parsed results into a DataFrame

In [80]:
# Extract business list
yelp_businesses = yelp_data.get("businesses", [])

#Create list of required fields from the yelp dict
yelp_list = [
    {
        "name": business.get("name"),
        "address": business.get("location", {}).get("address1", ""),
        "categories": ", ".join([category["title"] for category in business.get("categories", [])]),
        "rating": business.get("rating", None),
        "latitude": business.get("coordinates", {}).get("latitude"),
        "longitude": business.get("coordinates", {}).get("longitude")
    }
    for business in yelp_businesses
]

# Convert list to df
yelp_df = pd.DataFrame(yelp_list)

# Create file path for yelp df
yelp_file_path = "C:/Users/colte/LHL-final-project/data/yelp_df.csv"

# Add yelp_df.csv to file path
yelp_df.to_csv(yelp_file_path, index=False)

print(f"df was saved to {yelp_file_path}")

df was saved to C:/Users/colte/LHL-final-project/data/yelp_df.csv


# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating